## Installations


In [10]:
# !pip install langchain-groq
# %pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4
# !pip install shutup

## Imports

In [12]:

import os
import shutup
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.embeddings import HuggingFaceEmbeddings

shutup.please()

In [13]:
# Load data
loader = TextLoader("essay.txt")
docs = loader.load()

In [14]:

# Split text into chunks 
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

## GROQ

In [20]:
groq_key = 'your key'

In [16]:


# Define the embedding model
#using mixbread's embedding and in binary mode

embeddings = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1",
                                   encode_kwargs = {'precision': 'binary'})


# FAISS the vector store 
vector = FAISS.from_documents(documents, embeddings)

# Define a retriever interface
retriever = vector.as_retriever()

# Define LLM
model = ChatGroq(temperature=0, groq_api_key=groq_key, model_name="mixtral-8x7b-32768")

# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


## retrieved responses

In [17]:
%%time
response = retrieval_chain.invoke({"input": "where did fifa happened ?"})
print(response["answer"])

The 2022 FIFA World Cup was held in Qatar. It was the first World Cup to be held in the Arab world and Muslim world, and the second held entirely in Asia after the 2002 tournament in South Korea and Japan.
CPU times: user 88.3 ms, sys: 60.5 ms, total: 149 ms
Wall time: 754 ms
